In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from us_states import us_states

# Load Data

In [80]:
#Load Diabetes Rates and County Level Information
diabetes_rates_test = pd.read_csv('us_county_diabetes_rates.csv')
county_info = pd.read_csv('county_info.csv')

/var/folders/vk/kfflzmxs1ssf_t7swsphkgd00000gn/T/ipykernel_36425/8987213.py:3: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  county_info = pd.read_csv('county_info.csv')


***Percentage, lower limit, upper limit of adults aged 20+ diagnosed with diabetes. Data as of 2021 -- all counties except all of Florida and 8 other counties have 'No Data'.Going to impute previous year (2020) for 2021 data for all missing counties -- have already done it for Florida but not the 8 counties***

In [81]:
##Fix FIPS for Later use to join tables
### make them 5 string
diabetes_rates_test['FIPS'] = diabetes_rates_test['CountyFIPS'].astype(str).str.zfill(5)
diabetes_rates_test.drop(columns ='CountyFIPS')

,County,State,Percentage,Lower Limit,Upper Limit,FIPS
0,Clifton Forge City,Virginia,No Data,No Data,No Data,51560
1,Bedford City,Virginia,No Data,No Data,No Data,51515
2,Shannon County,South Dakota,No Data,No Data,No Data,46113
3,Washington County,Florida,No Data,No Data,No Data,12133
4,Walton County,Florida,No Data,No Data,No Data,12131
...,...,...,...,...,...,...
3146,Oglala Lakota County,South Dakota,15.4,12.7,18.4,46102
3147,Holmes County,Mississippi,15.5,11.1,19.5,28051
3148,Portsmouth City,Virginia,16.8,13.7,20.2,51740
3149,Williamsburg County,South Carolina,17.5,14.5,20.9,45089


In [82]:
## Replace all the No Data to NA
diabetes_rates_test = diabetes_rates_test.replace('No Data', np.nan)

In [83]:
## Prepping the florida section with all the 2020 rates (imputing)
florida_diabetes_2020 = pd.read_csv('flo_2020_diabetes.csv')

## Adjust FIPS for Easy Join
florida_diabetes_2020['FIPS'] = florida_diabetes_2020['CountyFIPS'].astype(str).str.zfill(5)


In [84]:
# dropped any counties that don't have percentage, UL, LL
no_nans = diabetes_rates_test.dropna(subset='Percentage').reset_index(drop = True)

In [85]:
#Combining the 2020 florida Data and 2021 data together (Make a full data)
complete_rates = pd.concat([no_nans, florida_diabetes_2020]).reset_index(drop = True)
complete_rates

# drop County FIPS (for later join)
complete_rates.drop(columns ='CountyFIPS')

,County,State,Percentage,Lower Limit,Upper Limit,FIPS
0,Teton County,Wyoming,4.4,3.1,6,56039
1,Gallatin County,Montana,4.7,3.5,6,30031
2,San Juan County,Washington,4.7,3.2,6.1,53055
3,Summit County,Colorado,5.1,3.5,7.2,08117
4,Blaine County,Idaho,5.1,3.6,7,16013
...,...,...,...,...,...,...
3138,Lafayette County,Florida,9.3,6.5,12.5,12067
3139,Lake County,Florida,11.0,9.0,13.4,12069
3140,Lee County,Florida,7.6,6.1,9.2,12071
3141,Leon County,Florida,8.3,6.3,10.5,12073


In [86]:
## Making all rates into floats (for Processing)
complete_rates['Percentage'] = complete_rates['Percentage'].astype(float)
complete_rates['Lower Limit'] = complete_rates['Lower Limit'].astype(float)
complete_rates['Upper Limit'] = complete_rates[' Upper Limit'].astype(float)


### County-Level Population and Income

In [87]:
#county_info only want population and Income
## Line Code 100 and 120 Respectively
county_pop_income = county_info[(county_info["LineCode"] == 100)|(county_info["LineCode"]== 120)].copy()
county_pop_income.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
7,"""00000""",United States,,CAINC30,100.0,...,Population (persons) 3/,Number of persons,201298000,203798722,...,319257560,321815121,324353340,326608609,328529577,330226227,331526933,332048977,333271411,334914895
9,"""00000""",United States,,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,3029,3176,...,29649,30578,31069,32314,33734,35035,35372,38554,40805,42715
30,"""01000""",Alabama,5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,3440000,3449846,...,4887757,4909177,4931579,4953096,4977088,5003778,5031864,5050380,5073903,5108468
32,"""01000""",Alabama,5,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,2236,2367,...,21886,22480,22833,23569,24452,25301,25848,27842,29718,31195
53,"""01001""","Autauga, AL",5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,25166,24606,...,56062,56311,56979,57393,57747,58251,58915,59203,59726,60342


In [88]:
#Drop GeoFips
county_pop_income['FIPS'] = county_pop_income['GeoFIPS'].astype(str).str.zfill(5)

county_pop_income = county_pop_income.drop(columns ='GeoFIPS', axis = 1)
county_pop_income

,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,1971,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,FIPS
7,United States,,CAINC30,100.0,...,Population (persons) 3/,Number of persons,201298000,203798722,206817509,...,321815121,324353340,326608609,328529577,330226227,331526933,332048977,333271411,334914895,"""00000"""
9,United States,,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,3029,3176,3342,...,30578,31069,32314,33734,35035,35372,38554,40805,42715,"""00000"""
30,Alabama,5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,3440000,3449846,3497452,...,4909177,4931579,4953096,4977088,5003778,5031864,5050380,5073903,5108468,"""01000"""
32,Alabama,5,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,2236,2367,2539,...,22480,22833,23569,24452,25301,25848,27842,29718,31195,"""01000"""
53,"Autauga, AL",5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,25166,24606,25508,...,56311,56979,57393,57747,58251,58915,59203,59726,60342,"""01001"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73540,Southwest,6.0,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,2673,2854,3005,...,29258,28715,30209,31927,33095,32623,35333,38627,40491,"""96000"""
73561,Rocky Mountain,7.0,CAINC30,100.0,...,Population (persons) 3/,Number of persons,4943000,5037873,5194044,...,11723574,11917655,12095615,12269211,12428025,12583415,12741331,12865778,12976939,"""97000"""
73563,Rocky Mountain,7.0,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,2724,2949,3163,...,29428,29391,30909,32811,34586,35626,38893,42108,44051,"""97000"""
73584,Far West,8.0,CAINC30,100.0,...,Population (persons) 3/,Number of persons,26635000,27101402,27570486,...,55208367,55738569,56173821,56477002,56663305,56772795,56471258,56414568,56374151,"""98000"""


In [89]:
## Turn GeoFIPs to FIPS (for later)

county_pop_income['FIPS'] = county_pop_income['FIPS'].str.replace('"', '')
county_pop_income['FIPS'] = county_pop_income['FIPS'].astype(str).str.strip()


## Get Specific columns
county_pop_income = county_pop_income[['FIPS','GeoName','Region','TableName','LineCode','IndustryClassification','Description','Unit','2020','2021','2022','2023']]

county_pop_income.head()

,FIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2020,2021,2022,2023
7,00000,United States,,CAINC30,100.0,...,Population (persons) 3/,Number of persons,331526933,332048977,333271411,334914895
9,00000,United States,,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,35372,38554,40805,42715
30,01000,Alabama,5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,5031864,5050380,5073903,5108468
32,01000,Alabama,5,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,25848,27842,29718,31195
53,01001,"Autauga, AL",5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,58915,59203,59726,60342


In [91]:
## Limit table to only county (no US)
county_pop_income = county_pop_income[county_pop_income['GeoName'].str.contains(',', na=False)].reset_index(drop=True)

#clean up string to get populations
mid = county_pop_income['GeoName'].str.rsplit(',',n = 1, expand = True).apply(lambda col: col.str.strip())
county_pop_income['County'] = mid[0].str.split('+')
county_pop_income['StateAbbr'] = mid[1].str.replace('*', '', regex=False)


## some of counties combined for some reason so expand
county_pop_income = county_pop_income.explode('County')
county_pop_income['County'] = county_pop_income['County'].astype(str)
county_pop_income.head()

,FIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2020,2021,2022,2023,County,StateAbbr
0,01001,"Autauga, AL",5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,58915,59203,59726,60342,Autauga,AL
1,01001,"Autauga, AL",5,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,27253,29300,31156,33640,Autauga,AL
2,01003,"Baldwin, AL",5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,233227,239439,246531,253507,Baldwin,AL
3,01003,"Baldwin, AL",5,CAINC30,120.0,...,Per capita net earnings 4/,Dollars,27734,30176,31729,33953,Baldwin,AL
4,01005,"Barbour, AL",5,CAINC30,100.0,...,Population (persons) 3/,Number of persons,24969,24533,24700,24585,Barbour,AL


In [97]:
#Combine together with rates
together = pd.merge(complete_rates,county_pop_income, on = 'FIPS')


#Columns of interest
official = together[['State','FIPS','County_x','Percentage','Lower Limit','Upper Limit','Upper Limit','Region','LineCode','IndustryClassification',	'Description','Unit','2020','2021','2022','2023','StateAbbr']]
official


,State,FIPS,County_x,Percentage,Lower Limit,Upper Limit,Upper Limit,Region,LineCode,IndustryClassification,Description,Unit,2020,2021,2022,2023,StateAbbr
0,Wyoming,56039,Teton County,4.4,3.1,6.0,6.0,7.0,100.0,...,Population (persons) 3/,Number of persons,23379,23605,23297,23232,WY
1,Wyoming,56039,Teton County,4.4,3.1,6.0,6.0,7.0,120.0,...,Per capita net earnings 4/,Dollars,71902,93615,95686,104988,WY
2,Montana,30031,Gallatin County,4.7,3.5,6.0,6.0,7,100.0,...,Population (persons) 3/,Number of persons,119597,123098,124938,126409,MT
3,Montana,30031,Gallatin County,4.7,3.5,6.0,6.0,7,120.0,...,Per capita net earnings 4/,Dollars,37116,43275,48024,51557,MT
4,Washington,53055,San Juan County,4.7,3.2,6.1,6.1,8.0,100.0,...,Population (persons) 3/,Number of persons,17826,18618,18672,18566,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6175,Florida,12071,Lee County,7.6,6.1,9.2,9.2,5,120.0,...,Per capita net earnings 4/,Dollars,22871,24245,25705,27483,FL
6176,Florida,12073,Leon County,8.3,6.3,10.5,10.5,5,100.0,...,Population (persons) 3/,Number of persons,293955,296007,297282,296913,FL
6177,Florida,12073,Leon County,8.3,6.3,10.5,10.5,5,120.0,...,Per capita net earnings 4/,Dollars,29668,32341,33589,35957,FL
6178,Florida,12075,Levy County,11.7,9.2,14.7,14.7,5,100.0,...,Population (persons) 3/,Number of persons,43059,44156,45253,46545,FL


In [98]:
#Make CSV of official
official.to_csv('county_pop_income_DR.csv', index=False)